# Training the Defence CNN

In this notebook, we shall train the defence CNN model (ran on just one epoch) with using two inputs. The first is the outputs from the autoencoder. These images are a mixture of non-adversary and adversary images. The second input is the binary label which indicates whether an image is adversary (1) or not (0). We decided not to use the binary classification probabilities as a feature as the adversary classifier was trained using the same dataset that would need to be classified to be used as a training set for the defence CNN. This could lead to systematic biases and we suspected the class distribution would differ between the training and test sets. If time allowed, we would have explored cross validation instead. We altered the model structure to adapt to accomodate these dual inputs.

Load necessary packages

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

In [2]:
NUM_CATEGORIES = 43

Load the outputs from the autoencoder, these can be found [here](https://drive.google.com/drive/u/0/folders/1tg24731_oseORhm1v2AMW-mOtC2wW8IJ)

In [3]:
bd_training_data = np.load('filter_denoised .npy')
black_patch_data = np.load('black_patched_denoised.npy')
white_patch_data = np.load('white_patched_training.npy')
noisy_data = np.load('noisy_train_denoised.npy')
same_patch_data = np.load('same_image_patched_denoised.npy')
bd_training_label = np.load('32_filter_training_labels.npy')
black_patch_label = np.load('32_filter_training_labels.npy')

Below we concatenate the image sets and also create a label array which holds the classes (0-42) for each of the images. Note, as the adversary images were created from the filter dataset, they are in the same order, hence we concatenate 4 of the 'black_patch_label' array with the filter training labels.

In [4]:
training_data = np.concatenate((bd_training_data,noisy_data,black_patch_data,white_patch_data,same_patch_data),axis=0,dtype='float32')
training_label = np.concatenate((bd_training_label,black_patch_label,black_patch_label,black_patch_label,black_patch_label),axis=0,dtype='float32')

Next we create our additional feature, which is a label 0 or 1 for each image, depending on whether it is adversary or not. The filter (untampered) images and the originally noisy images receive a 0 label and the patched adversaries receive a 1. We decided to label the noisy images with 0 for non-adversary as we assumed that the autoencoder will have removed any noise by this point.

In [6]:
ones = np.ones(117627,dtype='float32')
zeros = np.zeros(78418,dtype='float32')
adv_label = np.concatenate((zeros,ones),axis=0)

Create the train-validation split and shuffle.

In [7]:
X_train2, X_val2, y_train2, y_val2 = train_test_split(training_data, training_label, test_size=0.3, random_state=42, shuffle=True)
X_adv_train2, X_adv_val2, y_discard2, y_discard3 = train_test_split(adv_label, training_label, test_size=0.3, random_state=42, shuffle=True)

Here we one-hot-encode the class labels.

In [8]:
y_train2 = keras.utils.to_categorical(y_train2, NUM_CATEGORIES)
y_val2 = keras.utils.to_categorical(y_val2, NUM_CATEGORIES)


Here we define the architecture of our model, note the use of 2 inputs.

In [9]:
import tensorflow as tf
from tensorflow import keras

input1 = keras.layers.Input(shape=(32, 32, 3), name='pixel_input')
input2 = keras.layers.Input(shape=(1,), name='aux_input')
    
conv1 = keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(input1)
conv2 = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(conv1)
pool1 = keras.layers.MaxPool2D(pool_size=(2, 2))(conv2)
bn1 = keras.layers.BatchNormalization(axis=-1)(pool1)
    
conv3 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(bn1)
conv4 = keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu')(conv3)
pool2 = keras.layers.MaxPool2D(pool_size=(2, 2))(conv4)
bn2 = keras.layers.BatchNormalization(axis=-1)(pool2)
    
flatten = keras.layers.Flatten()(bn2)

merged = keras.layers.concatenate([flatten, input2])

dense1 = keras.layers.Dense(512, activation='relu')(merged)
bn3 = keras.layers.BatchNormalization()(dense1)
dropout = keras.layers.Dropout(rate=0.5)(bn3)

output = keras.layers.Dense(43, activation='softmax')(dropout)

model = keras.models.Model(inputs=[input1, input2], outputs=output)

In [10]:
lr = 0.001
epochs = 1

opt = tf.keras.optimizers.legacy.Adam(lr=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

C:\Users\Team Knowhow\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Here we augment the data to prevent overfitting and we fit the model to the training data.

In [11]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")
history2 = model.fit(aug.flow([X_train2,X_adv_train2], y_train2, batch_size=32), epochs=epochs, validation_data=([X_val2,X_adv_val2], y_val2))

4289/4289 [==============================] - 478s 110ms/step - loss: 0.6104 - accuracy: 0.8261 - val_loss: 0.1673 - val_accuracy: 0.9497


We see that after 1 epoch of training, the model has an accuracy score of 94.97% on the validation set. We then ran this model on the HPC for 200 epochs for better results. This scored an accuracy of 98.16% on the validation set. The model is saved in the Report/Models folder and is entitled 'DEFENCE_CNN.h5' We also trained a version of the Defence CNN without the classifier inputs in order to isolate the impacts of the autoencoder and the classifier. These results are discussed in the Model Comparison section.

In [13]:
model.save('defence_cnn_on_auto_1_epoch.h5')